<a href="https://colab.research.google.com/github/EthanBrewerCity/NLP_CW_210008199/blob/main/NLP_CW_210008199.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##NLP Project: Ethan Brewer <br>
Dataset: Dair-ai/emotion <br>
Project Type: Sentiment Analysis <br>


#Add Setup Features for Project <br>
This Includes device setup, installs, imports and loading the original dataset

In [26]:
# Installs necessary
!pip install transformers
!pip install datasets

In [27]:
# Add imports
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Further imports via pytorch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

# Imports from prior pip installs
from datasets import load_dataset
from transformers import DistilBertModel, DistilBertTokenizer, AdamW

# Sklearn imports
from sklearn.model_selection import learning_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load Dataset: dair-ai/emotion from Huggingface
dataset = load_dataset('emotion')

# Ensure the device being used is either the cuda or cpu (if cuda isn't available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Preprocessing the Data
The Tokeniser and Model are taken and the data is preprocessed via BERT (Currently Distilled)

In [28]:
# Load Tokeniser and Model for dataset, In this case, BERT (Distilled vers. may change later)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

## Create the Neural Network and Define the Model Architecture

In [29]:
# Defining Model Architecture
class SentiClass(nn.Module):
  def __init__(self,model):
    super(SentiClass, self).__init__()
    self.model = model
    self.linear = nn.Linear(768,1)
  def forward(self,input_ids, attention_mask):
    outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
    last_hidden_state = outputs.last_hidden_state[:, 0, :]
    logits = self.linear(last_hidden_state)
    return logits.squeeze(-1)

# Creatign intstance of the model
model = SentiClass(model)

## Define the Train/ Eval Loop

In [30]:
# Tokenize the data
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = dataset['train'].map(tokenize, batched=True)
test_dataset = dataset['test'].map(tokenize, batched=True)

# Set the data format
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.BCEWithLogitsLoss()

# Define the training loop
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, labels.float())
        print(loss)
        loss.backward()
        optimizer.step()

# Define the evaluation loop
def evaluate(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            predictions = torch.round(torch.sigmoid(outputs))

            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    return correct / total

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Pad the sequences
def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  attention_mask = [item['attention_mask'] for item in batch]
  labels = [item['label'] for item in batch]

  input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
  attention_mask = pad_sequence(attention_mask,batch_first=True,padding_value=0)

  return {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'label': torch.tensor(labels)
  }

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle = True, collate_fn = collate_fn)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle = False, collate_fn = collate_fn)

for epoch in range(5):
    train(model, train_loader, optimizer, loss_fn, device)
    accuracy = evaluate(model, test_loader, device)
    print(f'Epoch {epoch+1} - Test Accuracy: {accuracy:.3f}')

Streaming output truncated to the last 5000 lines.
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-0.4083, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-1.8107, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-0.1285, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-0.6505, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-1.8430, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-2.4606, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-3.0004, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-3.0487, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-2.4840, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(-4.3035, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
ten